In [1]:
import inro.modeller as m
mm = m.Modeller()
eb = mm.emmebank

create_exatt = mm.tool('inro.emme.data.extra_attribute.create_extra_attribute')

In [60]:
exatt_tag = "@sl"

create_exatt(extra_attribute_type = "LINK", 
            extra_attribute_name = exatt_tag,
            extra_attribute_default_value = 0,
            overwrite = True)

ExtraAttribute(@sl)

In [90]:
sc = eb.scenario(103)
sc.extra_attribute(exatt_tag).initialize(0)
ntwk = sc.get_network()
with open (file_dir + screenline_file +".csv") as slfile:
    lines = slfile.readlines()
    for line in lines:
        CurLine = line.rstrip("\r").split(",")
        i_node = int(CurLine[0])
        j_node = int(CurLine[1])
        if ntwk.link(i_node, j_node):  
            link = ntwk.link(i_node, j_node)
            link[exatt_tag] = 1
sc.publish_network(ntwk)

In [154]:

screenline_file = "Midest_Southwest_Links_IN"

file_dir = "C:\\Users\\mnasterska\\Documents\\Projects\\GGH\\MOE\\Screenlines\\"
#file_dir = "M:\\10039241 - GGH Multimodal Transportation\\Screenlines\\"

In [155]:
sc_num = 109
sc = eb.scenario(sc_num)

ntwk = sc.get_network()

capacity = {"persons": 0.0,
            "auto": 0.0,
            "train": 0.0,
            "subway": 0.0,
            "bus": 0.0
           }
volume = {"persons": 0.0,
            "auto": 0.0,
            "train": 0.0,
            "subway": 0.0,
            "bus": 0.0
           }
delay = { "subway": 0.0,
            "bus": 0.0
           }
count = {"subway": 0,
            "bus": 0
           }

sov_vols = 0.0
hov2_vols = 0.0
hov3_vols = 0.0


with open (file_dir + screenline_file +".csv") as slfile:
    lines = slfile.readlines()
    for line in lines:
        CurLine = line.rstrip("\r").split(",")
        i_node = int(CurLine[0])
        j_node = int(CurLine[1])
        if ntwk.link(i_node, j_node):  
            link = ntwk.link(i_node, j_node)

            sov_link = link["@volau1_nt"] + link["@volau1_toll"]
            hov2_link = link["@volau2_nt_hov"] + link["@volau2_nt_nh"] + link["@volau2_toll_hov"] + link["@volau2_toll_nh"]
            hov3_link= link["@volau3_nt_hov"] + link["@volau3_nt_nh"] + link["@volau3_toll_hov"] + link["@volau3_toll_nh"]
            truck_link = link["@volau_htruck"] + link["@volau_mtruck"] +link["@volau_ltruck"]
            sov_vols += sov_link
            hov2_vols += hov2_link
            hov3_vols += hov3_link
            if ntwk.mode("I") in link.modes:
                volume["auto"] += (sov_link + hov2_link + hov3_link + truck_link)
                capacity["auto"] += link.data3 * link.num_lanes
            volume["persons"]+= sov_link + hov2_link*2 + hov3_link*3.3
            
            for transit in link.segments():

                #calculate average redution in speed for transit lines crossing screeline
                # timau+length*us2
                mode = transit.line.mode

                if mode != "r":
                    cong_line = transit.line
                    #print cong_line.id
                    ff_time = 0.0
                    cong_time = 0.0
                    length = 0.0
                    for segment in cong_line.segments():
                        length += segment.link.length
                        if segment.transit_time_func == 2 or segment.transit_time_func ==3 :
                            #cong_time += segment.transit_time
                            cong_time += segment.link.auto_time + segment.link.length*segment.data2
                            ff_au_time  = (segment.link.length*60/segment.link.data2)
                            ff_time += ff_au_time + segment.link.length*segment.data2
                        else:
                            cong_time += segment.transit_time
                            ff_time += segment.transit_time
                    #print "congested time " + str(cong_time) + " speed " + str(length*60/cong_time)
                    #print "free flow time " + str(ff_time)  + " speed " + str(length*60/ff_time)
                    diff = length*60/ff_time - length*60/cong_time
                #add MTO peak-hour factors
                factor = 1.0
                if mode.id == 'r':
                    factor = 0.75
                elif mode.id == "m" or mode.id == "l":
                    factor = 0.59
                elif mode.id == "g" or mode.id == "p":
                    factor = 0.57
                elif mode.id == "b" or mode.id == "s":
                    factor = 0.64
                elif mode.id == "q" or mode.id == "n":
                    factor = 0.55
                else:
                    print "Mode for segment %s not found!"%(transit)
                veh = transit.line.vehicle
                voltr = transit['@voltr_avg'] * factor
                captr = transit.line['@capacity_total']/2
                #print transit, captr, voltr
                #person_vols += voltr
                
                volume["persons"] += voltr
                if mode.id == 'r':
                    volume["train"] += voltr
                    capacity["train"] +=captr
                elif mode.id == "m" or mode.id == "l":
                    volume["subway"] += voltr
                    capacity["subway"] +=captr
                    delay["subway"] += diff
                    count["subway"] += 1
                else:
                    volume["bus"] += voltr
                    capacity["bus"] +=captr
                    delay["bus"] += diff
                    count["bus"] += 1
                #if not veh_capacity.has_key(veh):
                #    veh_capacity[veh] = captr
                #else:
                #    veh_capacity[veh] += captr
                #if not veh_volumes.has_key(veh):
                #    veh_volumes[veh] = voltr
                #else:
                #    veh_volumes[veh] += voltr
                

transit_persons = volume["train"] + volume["subway"] + volume["bus"]
print "persons, %s, %s"%(volume["persons"],capacity["persons"])
print "transit trips, %s"%(transit_persons)
print "vehicle, %s, %s"%(volume["auto"],capacity["auto"])
print "train, %s, %s"%(volume["train"],capacity["train"])
print "subway/lrt, %s, %s"%(volume["subway"],capacity["subway"])
print "bus, %s, %s"%(volume["bus"],capacity["bus"])
print "sov, ",sov_vols
print "hov2, ",hov2_vols
print "hov3, ",hov3_vols
#print "avg suwbay delay %s km/hr"%(delay["subway"]/count["subway"])
#print "avg bus delay %s km/hr"%(delay["bus"]/count["bus"])


persons, 17876.8134888, 0.0
transit trips, 915.663574595
vehicle, 17751.3954927, 28200.0
train, 250.637784004, 3200.0
subway/lrt, 0.0, 0.0
bus, 665.025790591, 1035.64060974
sov,  13054.0855402
hov2,  1663.80971851
hov3,  175.589374853


In [19]:
print set([ntwk.mode("I")]) in 

set([Mode(I)])
